In [1]:
try:
    import requests
except ImportError:
    print('Please install "requests" module: pip install requests')
    raise

import csv
import traceback

In [2]:
def GET(*args, **kwargs):
    hdrs = kwargs.setdefault('headers', {})
    hdrs['User-Agent'] = 'github.com/denilsonsa/cartorios-autorizados-apostila'
    return requests.get(*args, **kwargs)

In [3]:
BASE = 'http://www.cnj.jus.br/seiapostila/controle.php?'

def get_states():
    r = GET(BASE + 'action=estado')
    data = r.json()
    return data

def get_cities(state_id):
    r = GET(BASE + 'action=cidade&idEstado={0}'.format(state_id))
    data = r.json()
    return data

def get_offices(city_id):
    r = GET(BASE + 'action=unidade&idCidade={0}'.format(city_id))
    data = r.json()
    return data

def get_office_details(office_id):
    r = GET('http://www.cnj.jus.br/corregedoria/justica_aberta/?' +
            'd=consulta_extra&a=consulta_extra&f=seiApostila&cns={0}'.format(office_id))
    data = r.json()
    return data

In [4]:
def get_all_office_details():
    for state in get_states():
        # state has this format:
        # {"id":"1","sigla":"RO","estado":"Rond\u00f4nia"}
        for city in get_cities(state['id']):
            # city has this format:
            # {"id":"17","sigla":"17","cidade":"Porto Velho"}
            print('Parsing {0} - {1}'.format(state['estado'], city['cidade']))
            for office in get_offices(city['id']):
                # office has this format:
                # {"cns":"000786","cartorio":"2\u00ba Tabelionato de Notas"}
                details = None
                try:
                    details = get_office_details(office['cns'])
                    # details has this format:
                    # {
                    #    "CNS" : "00.078-6",
                    #    "COD_IDENTIFICACAO_UNICA" : "000786",
                    #    "DAT_INSTALACAO" : "1966-08-12",
                    #    "DSC_BAIRRO_SERVENTIA" : "Quinze",
                    #    "DSC_CEP_SERVENTIA" : "69901230",
                    #    "DSC_COMPLEMENTO" : "",
                    #    "DSC_DENOMINACAO_SERVENTIA" : "2º Tabelionato de Notas",
                    #    "DSC_DISTRITO" : "",
                    #    "DSC_EMAIL_SERVENTIA" : "setan2rb@tj.ac.gov.br",
                    #    "DSC_END_SERVENTIA" : "Boulevard Augusto Monteiro, n. 810"
                    #    "DSC_SUB_DISTRITO" : "",
                    #    "DS_CIDADE" : "RIO BRANCO",
                    #    "FLG_STATUS" : "0",
                    #    "IND_TIPO_SITUACAO" : "1",
                    #    "IND_UF" : "AC",
                    #    "NUM_CNPJ_SERVENTIA" : "05396544000138",
                    #    "NUM_FAX_SERVENTIA" : "2147483647",
                    #    "SEQ_DADOS_SERVENTIA" : "937",
                    #    "atribuicoes" : "Notas",
                    #    "decisao_atual" : "PROVIDO",
                    #    "num_telefone_serventia" : "6832115497",
                    #    "num_telefone_serventia_2" : "",
                    #    "responsavel" : "Gilcineide Ribeiro Batista",
                    # }
                    details['estado'] = state['estado']
                    details['estado_id'] = state['id']
                    details['estado_sigla'] = state['sigla']
                    details['cidade'] = city['cidade']
                    details['cidade_id'] = city['id']
                    yield details
                except:
                    print('\n'.join([
                        'Ignoring an exception!',
                        'state={0!r}',
                        'city={1!r}',
                        'office={2!r}',
                        'details={3!r}',
                        'exception={4}',
                    ]).format(
                        state, city, office, details, traceback.format_exc()
                    ))

In [5]:
def write_csv_file(filename, rows):
    with open(filename, 'w') as csvfile:
        writer = None
        for row in rows:
            if writer is None:
                fieldnames = sorted(row.keys())
                writer = csv.DictWriter(
                    csvfile, fieldnames, extrasaction='ignore'
                )
                writer.writeheader()
            writer.writerow(row)

In [6]:
write_csv_file('cartorios-autorizados-apostila.csv', get_all_office_details())

Parsing Rondônia - Porto Velho
Parsing Acre - Rio Branco
Parsing Amazonas - Manaus
Parsing Roraima - Boa Vista
Parsing Pará - Belém
Parsing Amapá - Macapá
Parsing Tocantins - Palmas
Parsing Maranhão - São Luís
Parsing Piauí - Teresina
Parsing Ceará - Fortaleza
Parsing Rio Grande do Norte - Natal
Parsing Paraíba - João Pessoa
Parsing Pernambuco - Recife
Parsing Alagoas - Maceió
Parsing Sergipe - Aracaju
Parsing Bahia - Salvador
Parsing Minas Gerais - Belo Horizonte
Parsing Espírito Santo - Vitória
Parsing Rio de Janeiro - Rio de Janeiro
Parsing São Paulo - São Paulo
Parsing Paraná - Curitiba
Parsing Santa Catarina - Florianópolis
Parsing Rio Grande do Sul - Porto Alegre
Ignoring an exception!
state={'id': '23', 'estado': 'Rio Grande do Sul', 'sigla': 'RS'}
city={'id': '4927', 'cidade': 'Porto Alegre', 'sigla': '4927'}
office={'cns': 'TESTE', 'cartorio': 'Unidade de Teste'}
details=None
exception=Traceback (most recent call last):
  File "<ipython-input-4-7e2fd3e1a04d>", line 41, in get_